In [11]:
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from torchvision.transforms import transforms
from tqdm import tqdm

In [12]:
import torch
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()

        self.features = nn.Sequential(
           #ConvolutionLayers
           ##1
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(4,stride = 4),
            ##2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,stride = 2),
            ##3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AvgPool2d(8,stride = 8),
            nn.Flatten(1),
            ##Classifier 
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,148),
            nn.ReLU(),
            nn.Linear(148,2)
        )

    def forward(self, x):
      x = self.features(x)

      return x

core_model = CustomCNN()
random_input = torch.randn(1, 1, 160, 160)
output = core_model(random_input)
print("\nCNN Output:")
print(output)



CNN Output:
tensor([[0.0028, 0.0257]], grad_fn=<AddmmBackward0>)


In [13]:
from torch.utils.data import Dataset
import os
from glob import glob
import natsort
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

class CoreDataset(Dataset):
    def __init__(self, folder_paths, transform=None, train=True):

        subset_numbers = {
            'I': [1, 7, 8, 9, 36, 37],
            'II': [2, 5, 10, 11, 12, 13, 15, 39, 40, 41, 42, 44],
            'III': [3, 6, 14, 16, 17, 19, 20, 43, 45, 46, 48, 49],
            'IV': [18, 21, 22, 23, 24, 25, 26, 47, 50, 51, 52, 53, 54, 55, 38],
            'V': [4, 27, 28, 29, 30, 31, 32, 33, 34, 35, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]
        }
        i = 0
        self.folder_paths = folder_paths
        self.transform = transform
        self.train = train
        self.train_dataset = []
        self.test_dataset = []
        self.train_dataset_lables = []
        self.test_dataset_lables = []
        self.file_names = []
        self.file_path = []
        label_to_value = {"I": 0, "II": 1, "III": 2,"IV": 3,"V": 4}
        for folder_path in folder_paths:
            #print(folder_path)
            for filename in natsort.natsorted(glob(os.path.join(folder_path, '*'))):
                    self.file_names.append((filename.split('/')[-1]).split('\\')[-1])
                    self.file_path.append(filename)
            #print(file_path)
            for i in range(len(self.file_names)):
                person_number = int(''.join(filter(str.isdigit, self.file_names[i])))
                for subset_name, subset_numbers_list in subset_numbers.items():
                    if label_to_value[subset_name] in (1, 2, 3, 4) and (self.train==True):
                        #print("train")
                        self.train_dataset.append(self.file_path[i])
                        if "6P" in str(folder_path):
                            self.train_dataset_lables.append(0)
                        elif "7P" in str(folder_path):
                            self.train_dataset_lables.append(1)
                    elif label_to_value[subset_name] == 1 and (self.train==False):
                        #print("test")
                        self.test_dataset.append(self.file_path[i])
                        if "6P" in str(folder_path):
                            self.test_dataset_lables.append(0)
                        elif "7P" in str(folder_path):
                            self.test_dataset_lables.append(1)
            
        #return train_dataset, train_dataset_lables, test_dataset, test_dataset_lables
    def __len__(self):
        if self.train == True :
            return len(self.train_dataset)
        else: 
            return len(self.test_dataset)
    def __getitem__(self, idx):
        if self.train:
            image = Image.open(self.train_dataset[idx]).convert("L")
            label = self.train_dataset_lables[idx]
        else:
            image = Image.open(self.test_dataset[idx]).convert("L")
            label = self.test_dataset_lables[idx]

        if self.transform is not None:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5), 
    transforms.RandomRotation(degrees=15),
    #transforms.RandomPerspective(distortion_scale=0.5, p=0.5)
    #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),

])


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 0.001
epochs = 5
folder_paths = ["C:/Users/pariya/OneDrive/Desktop/A_project/YaleB/B06P00", "C:/Users/pariya/OneDrive/Desktop/A_project/YaleB/B07P00"]


core_model = CustomCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(core_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)
train_dataset = CoreDataset(folder_paths, transform=transform, train = True)
test_dataset = CoreDataset(folder_paths, transform=transform, train = False)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)


In [15]:
import plotly.graph_objects as go
import plotly.offline as pyo

train_losses = []
test_losses = []


for epoch in range(epochs):
    core_model.train()
    total_train_loss = 0.0

    for inputs, labels in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs} - Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = core_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    average_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(average_train_loss)


# plot the loss

trace_train = go.Scatter(
    x=list(range(1, epochs + 1)),
    y=train_losses,
    mode='lines+markers',
    name='Train Loss'
)

layout = go.Layout(
    title='Training Losses Over Epochs',
    xaxis=dict(title='Epoch'),
    yaxis=dict(title='Loss'),
    hovermode='closest'
)
fig = go.Figure(data=[trace_train], layout=layout)
pyo.iplot(fig)

In [16]:
core_model.eval()

predictions = []
labels = []

with torch.no_grad():
    for test_inputs, test_labels in tqdm(test_dataloader, desc="Testing", leave=False):
        test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
        test_outputs = core_model(test_inputs)
        prediction = torch.argmax(test_outputs, dim=1).cpu().numpy()
        predictions.extend(prediction)
        labels.extend(test_labels.cpu().numpy())


f1 = f1_score(labels, predictions)
accuracy = accuracy_score(labels, predictions)


print(f"F1 Score = ", f1)
print(f"Accuracy = ", accuracy)

F1 Score =  0.7924528301886793
Accuracy =  0.6615384615384615
